In [ ]:
import requests
import pandas as pd
import mysql.connector
from datetime import datetime

table_name = 'radio_data' #this is table name
df = pd.read_csv('station_links.csv', usecols =[0], squeeze = True) #reading links from file here
print("Total no of URLs: {}".format(len(df.tolist()))) #print total no of links in file

In [ ]:
#Establishing connection with database 
def connect_with_db():
    db = mysql.connector.connect(
        host = "your-ip",
        database = "your-db-name",
        user = "your-user-name",
        password = "your-db-password",
        auth_plugin = "mysql_native_password",
    )
    if db:
        print('Connection Established')
        cursor = db.cursor()
        cursor.execute('SELECT DATABASE()')
        print('Current Database Name : ',cursor.fetchall()[0][0])
        return db
    else:
        print('Connection Denied')
        return db
mydb = connect_with_db()

In [ ]:
def create_table(table_name,mydb):
    try:
        if mydb:
            cursor = mydb.cursor()
            create_table_sql = "CREATE TABLE {} (\
                                    data_id BIGINT PRIMARY KEY AUTO_INCREMENT NOT NULL, \
                                    source_type VARCHAR(255) NOT NULL, \
                                    station_name VARCHAR(255) NOT NULL , \
                                    artist_name VARCHAR(255) UNIQUE KEY NOT NULL, \
                                    song_title VARCHAR(255) UNIQUE KEY NOT NULL, \
                                    listeners INT NOT NULL, \
                                    date_time DATETIME NOT NULL)".format(table_name)
            
            cursor.execute(create_table_sql)
            print('[INFO..] Table {} Created Successfully.'.format(table_name))
        else:
            print('Connection Denied')

    except Exception as E:
        print(E)
        
#if you want to create table in database run this fuction
create_table(table_name,mydb)

In [ ]:
def insert_data(table_name, count, *argv):
    if mydb:
        cursor = mydb.cursor() 
        cursor.execute('SELECT MAX(data_id) FROM radio_data;')
        last_id = cursor.fetchall()[0][0]
        if last_id == None:
            count = 0
        else:
            count = last_id
        count += 1
        insert_sql = 'INSERT INTO {} (data_id , source_type, station_name, artist_name, song_title, listeners, date_time) VALUES ({}, "{}", "{}", "{}", "{}", {}, "{}")'.format(table_name,count,argv[0],argv[1],argv[2],argv[3],argv[4],argv[5])
        cursor.execute(insert_sql)
        mydb.commit()
    else:
        print('Connection denied')

def parse_data(dic):
    date_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    a_name , s_name = dic['streams'][0]['songtitle'].split(' - ')
    source_type = dic['streams'][0]['content'].replace('\\/','/')
    return source_type, dic['streams'][0]['servertitle'], a_name, s_name, dic['currentlisteners'], date_time

In [ ]:
count = 0
while(True):
    for i,URL in enumerate(df.tolist()):
        try:
            dic = eval(requests.get(URL).text)
            try:
                song_type, server_name, artist_name, song_name, listeners, time_stamp = parse_data(dic)
                insert_data(table_name, count, song_type, server_name, artist_name, song_name, listeners, time_stamp)
                print('\n[INFO] Inserted Sucessfully.\n')
            except Exception as E:
                #print('```\n', E, '\n```')
                continue
            count += 1 
        except Exception as E:
            #print('~~~\n', E, '\n~~~')            
            continue


In [ ]:
#you should not run this it will delete your table in database, it is just for debugging purpose
def delete_table(table_name):
    try:
        if mydb:
            cursor = mydb.cursor()
            delete_table_sql = 'DROP TABLE {}'.format(table_name)
            cursor.execute(delete_table_sql)
            print('[INFO..] Table {} Successfull Deleted...'.format(table_name))
        else:
            print('Connection Failed')
    except Exception as E:
        print(E)

# delete_table(table_name)